# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [14]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [15]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    #TODO 1: Add your code here
    ## Read the data
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    
    ## Add the "Original language"
    df_eng["Original Language"] = "English"
    df_fr["Original Language"] = "French"
    df_sp["Original Language"] = "Spanish"
    
    ## rename column names
    colnames = df_eng.columns
    df_fr.columns = colnames
    df_sp.columns = colnames
    
    ## Contatenate
    df = pd.concat([df_eng, df_fr, df_sp])
    
    return df

df = preprocess_data()

In [16]:
df.sample(10)

,Movie / TV Series,Year,Synopsis,Review,Original Language
0,La La Land,2016,Cette comédie musicale raconte l'histoire d'un...,"""La La Land est un film absolument magnifique ...",French
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English
7,Astérix aux Jeux Olympiques,2008,Dans cette adaptation cinématographique de la ...,"""Ce film est une déception totale. Les blagues...",French
6,Toc Toc,2017,"En esta comedia española, un grupo de personas...","""Toc Toc es una película aburrida y poco origi...",Spanish
8,Les Visiteurs en Amérique,2000,Dans cette suite de la comédie française Les V...,"""Le film est une perte de temps totale. Les bl...",French
1,La Casa de Papel,(2017-2021),Esta serie de televisión española sigue a un g...,"""La Casa de Papel es una serie emocionante y a...",Spanish
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English
9,El Incidente,2014,"En esta película de terror mexicana, un grupo ...","""El Incidente es una película aburrida y sin s...",Spanish
9,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...","""Ce film est un gâchis complet. Les personnage...",French


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [17]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt")

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

In [18]:
## Test whether translate function works
import warnings
warnings.filterwarnings('ignore')

text = "hoy es navidad"
translated = translate(text, es_en_model, es_en_tokenizer)
print(translated)

Today's Christmas.


In [19]:
es_reviews = df.loc[df["Original Language"] == "Spanish", "Review"]
type(es_reviews.tolist())

list

In [20]:
# TODO 4: Update the code below

# # filter reviews in French and translate to English
fr_reviews = df.loc[df["Original Language"] == "French", "Review"].tolist()
fr_reviews_en = []
for review in fr_reviews:
    fr_reviews_en.append(translate(review, fr_en_model, fr_en_tokenizer))

# # filter synopsis in French and translate to English
fr_synopsis = df.loc[df["Original Language"] == "French", "Synopsis"].tolist()
fr_synopsis_en = []
for synopsis in fr_synopsis:
    fr_synopsis_en.append(translate(synopsis, fr_en_model, fr_en_tokenizer))

# filter reviews in Spanish and translate to English
es_reviews = df.loc[df["Original Language"] == "Spanish", "Review"].tolist()
es_reviews_en = []
for review in es_reviews:
    es_reviews_en.append(translate(review, es_en_model, es_en_tokenizer))

# # filter synopsis in Spanish and translate to English
es_synopsis = df.loc[df["Original Language"] == "Spanish", "Synopsis"].tolist()
es_synopsis_en = []
for synopsis in es_synopsis:
    es_synopsis_en.append(translate(synopsis, es_en_model, es_en_tokenizer))

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data

In [21]:
df.loc[df["Original Language"] == "French", "Review"] = fr_reviews_en
df.loc[df["Original Language"] == "French", "Synopsis"] = fr_synopsis_en
df.loc[df["Original Language"] == "Spanish", "Review"] = es_reviews_en
df.loc[df["Original Language"] == "Spanish", "Synopsis"] = es_synopsis_en

In [22]:
display(df.sample(10))

,Movie / TV Series,Year,Synopsis,Review,Original Language
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English
5,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",Spanish
0,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,"""Rome is a beautiful and moving film that pays...",Spanish
2,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",Spanish
6,Toc Toc,2017,"In this Spanish comedy, a group of people with...","""Toc Toc is a boring and unoriginal film that ...",Spanish
1,Intouchables,2011,This film tells the story of the unlikely frie...,"""Untouchables is an incredibly touching film w...",French
9,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",French
7,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",Spanish
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [23]:
from transformers import pipeline

In [24]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "siebert/sentiment-roberta-large-english"
sentiment_classifier = pipeline("sentiment-analysis", model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    return classifier(text)

In [25]:
## This model seems a bit inaccurate
analyze_sentiment("I hate this", sentiment_classifier)[0]["score"]

0.9994506239891052

In [26]:
analyze_sentiment("I love this", sentiment_classifier)[0]["score"]

0.9988621473312378

In [27]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
df["Sentiment"] = df["Review"].apply(lambda x: "Positive" if analyze_sentiment(x, sentiment_classifier)[0]["score"] >= 0.999 else "Negative")

In [28]:
display(df.sample(30))

,Movie / TV Series,Year,Synopsis,Review,Original Language,Sentiment
4,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,"""The Fabulous Destiny of Amélie Poulain is an ...",French,Negative
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English,Negative
4,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",Spanish,Negative
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English,Negative
3,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...","""The Labyrinth of Fauno is a fascinating and e...",Spanish,Negative
1,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",Spanish,Negative
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English,Negative
2,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",Spanish,Negative
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English,Positive
0,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,"""Rome is a beautiful and moving film that pays...",Spanish,Negative


In [16]:
# export the results to a .csv file
df.to_csv("reviews_with_sentiment.csv", index=False)